# Note: currently not using names to filter the keywords

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import copy
from matplotlib.ticker import PercentFormatter

import clip
import pickle
import torch
import re
import csv

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
with open('../data_files/openwebtext-clean2.num') as f:
    lines = f.readlines()

### Get names from predefines text files

In [4]:
# get all names (first and last) in a list:
with open('../random-name/first-names.txt') as f:
    namelines = f.readlines()
with open('../random-name/middle-names.txt') as f:
    mnamelines = f.readlines()
name_list = []
for line in namelines:
    name_list.append(line[:-1].lower())
for line in mnamelines:
    name_list.append(line[:-1].lower())
print(len(name_list))

8842


In [5]:
# get all words in a list
freq_list_noname = []
freq_list = []
name_set = set(name_list)

for line in lines:
    splitline = line.split("\t")
    middl = list(map(int,splitline[1:3]))

    newline = [splitline[0].strip("''").lower()] + middl + [int(splitline[4])]    
    ascii_word = re.sub(r'[^\w\s]','',newline[0])
    if len(ascii_word) == 0 :
        continue
    freq_list.append(newline)
    if newline[0] not in name_set:
        freq_list_noname.append(newline)

print(len(freq_list)) 
print(len(freq_list_noname)) 

2092966
2083622


In [6]:
freq_list.sort(key=lambda row:row[2])
freq_list.reverse()
top100th = freq_list[:100000]
top100th[:10]

[['the', 445503354, 444364108, 7958752],
 ['to', 231470564, 231003503, 7933619],
 ['p', 234522618, 228609674, 8424304],
 ['and', 205912374, 205088775, 7978227],
 ['of', 205778822, 203996377, 7706269],
 ['a', 188263522, 187046991, 7920297],
 ['in', 148481168, 147433279, 7612827],
 ['that', 103273639, 102490626, 6922986],
 ['is', 98127345, 97226904, 7285873],
 ['for', 83179488, 82242810, 7465139]]

In [7]:
# check the ascii coversion went right
for line in top100th:
    ascii_word = re.sub(r'[^\w\s]','',line[0])
    if len(ascii_word) == 0 :
        print("hey", line[0])

## Remove keywords with more than 4 tokens:

In [8]:
from transformers import  CLIPTokenizerFast
tokenizer =  CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
top100th_small = []
for line in top100th:
    word = line[0]
    toks = tokenizer(word).input_ids[1:-1]
    if len(toks)>4:
        continue
    top100th_small.append(line)

In [10]:
# save the top 10 thousand most common words
top10th = top100th_small[:10000]
print(len(top10th))
f = open("../data_files/openwebtext_top10th.txt", "w")
f.write(str(top10th))
f.close()


10000


In [11]:
print("First 100 words:", [word[0] for word in top10th[:100]])

First 100 words: ['the', 'to', 'p', 'and', 'of', 'a', 'in', 'that', 'is', 'for', 'it', 'on', 'i', 'with', 'you', 'this', 'as', 's', 'be', 'are', 'was', 'have', 'not', 'at', 'by', 'but', 'from', 'we', 'he', 'they', 'h', 'or', 'an', 's', 'has', 'will', 'can', 'his', 'more', 'if', 'all', 'about', 'your', 'their', 'one', 'who', 't', 'there', 'so', 'what', 'said', 'do', 'would', 'out', 'when', 'up', 'which', 'like', 'been', 'were', 'just', 'our', 'people', 'had', 'no', 'my', 'new', 'some', 'also', 'other', 'time', 'than', 'them', 'get', 'how', 'its', "n't", 'into', 'only', 'after', 'now', 'any', 'because', 'me', 'over', 'could', 'these', 'first', 'she', 'even', 'us', 'most', 'her', 'make', 'then', 'two', 'years', 'may', 'see', 'many']
